In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

np.random.seed(1337)
tf.random.set_seed(1337)

# Data

In [2]:
# Load the dataframes
train_data = pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

# Preview the dataframes
display(train_data.head(10))
display(test_data.head(10))

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
5,IWHA,0
6,GEFV,0
7,KARS,0
8,IMGW,0
9,KYRK,0


,Sequence
0,HWFK
1,MWPW
2,ALDV
3,NTLG
4,LHYY
5,AFGM
6,TPNY
7,EAKD
8,NDKE
9,GFIS


In [3]:
def create_char_df(df):
    """Create a dataframe that contains four columns (one for each char in 'Sequence')."""

    # Split sequences into characters
    first = []
    second = []
    third = []
    fourth = []

    for _, row in df.iterrows():
        ch_1, ch_2, ch_3, ch_4 = row["Sequence"]

        first.append(ch_1)
        second.append(ch_2)
        third.append(ch_3)
        fourth.append(ch_4)

    df_dict = {
        "first": first,
        "second": second,
        "third": third,
        "fourth": fourth
    }
    return pd.DataFrame(df_dict)

X_df = create_char_df(train_data)
X_test_df = create_char_df(test_data)

display(X_df.head(5))
display(X_test_df.head(5))

,first,second,third,fourth
0,D,K,W,L
1,F,C,H,N
2,K,D,Q,P
3,F,N,W,I
4,N,K,R,M


,first,second,third,fourth
0,H,W,F,K
1,M,W,P,W
2,A,L,D,V
3,N,T,L,G
4,L,H,Y,Y


## OneHot Encoding

In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_df)

display(enc.categories_)

# Ensure the onehots are the same for each letter
assert((enc.categories_[0] == enc.categories_[1]).all() and (enc.categories_[1] == enc.categories_[2]).all() and (enc.categories_[2] == enc.categories_[3]).all())

[array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)]

# Training

In [5]:
from sklearn.model_selection import train_test_split

X = enc.transform(X_df).toarray()
y = train_data["Active"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.02, random_state=1337, stratify=y)

# Create training np arrays
#X_train = enc.transform(X_df).toarray().reshape(-1, 4, 20)
#y_train = train_data["Active"].values

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GRU, Bidirectional, Dropout
import keras
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight('balanced', np.unique(y_train), y_train)
class_weights = {
    0: class_weight[0] / np.sum(class_weight),
    1: class_weight[1] / np.sum(class_weight)
}

batch_size = 128

model = Sequential()
model.add(Dense(120, activation = 'relu', input_dim=80))
model.add(Dropout(0.2))
model.add(Dense(40, activation = 'relu'))
#model.add(Dense(80, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=[f1_m]
)

history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val))
print(model.summary())

Epoch 1/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0933 - f1_m: 0.3334 - val_loss: 0.0479 - val_f1_m: 0.6638
Epoch 2/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0430 - f1_m: 0.7293 - val_loss: 0.0291 - val_f1_m: 0.7883
Epoch 3/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0324 - f1_m: 0.8003 - val_loss: 0.0228 - val_f1_m: 0.8600
Epoch 4/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0275 - f1_m: 0.8258 - val_loss: 0.0209 - val_f1_m: 0.8415
Epoch 5/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0242 - f1_m: 0.8513 - val_loss: 0.0178 - val_f1_m: 0.8999
Epoch 6/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0213 - f1_m: 0.8691 - val_loss: 0.0176 - val_f1_m: 0.9016
Epoch 7/50
858/858 [==============================] - 2s 2ms/step - loss: 0.0202 - f1_m: 0.8755 - val_loss: 0.0156 - val_f1_m: 0.9008
Epoch 8/50
858/858 [==============================] - 2s 2ms/s

In [ ]:
from sklearn.metrics import f1_score
y_pred = np.around(model.predict(X_val))
f1_score(y_val, y_pred)

In [ ]:
X_test = enc.transform(X_test_df).toarray()

In [ ]:
y_test=np.around(model.predict(X_test))

In [ ]:
np.savetxt("submission.txt", y_test, fmt="%d")

In [ ]:
display(y_test)